## Daemon thread in Java
Daemon thread is a low priority thread that runs in background to perform tasks such as garbage collection.

Properties:
1. They can not prevent the JVM from exiting when all the user threads finish their execution.
2. JVM terminates itself when all user threads finish their execution
3. If JVM finds running daemon thread, it terminates the thread and after that shutdown itself. JVM does not care whether Daemon thread is running or not.
4. It is an utmost low priority thread.

1.
```java
// Java program to demonstrate the usage of setDaemon() and isDaemon() method.
public class DaemonThread extends Thread{
    public DaemonThread(String name){
        super(name);
    }
 
    public void run(){ 
        // Checking whether the thread is Daemon or not
        if(Thread.currentThread().isDaemon()) { 
            System.out.println(getName() + " is Daemon thread"); 
        }else{ 
            System.out.println(getName() + " is User thread"); 
        } 
    } 
     
    public static void main(String[] args){
        DaemonThread t1 = new DaemonThread("t1");
        DaemonThread t2 = new DaemonThread("t2");
        DaemonThread t3 = new DaemonThread("t3");
     
        // Setting user thread t1 to Daemon
        t1.setDaemon(true);
             
        // starting first 2 threads 
        t1.start(); 
        t2.start();
 
        // Setting user thread t3 to Daemon
        t3.setDaemon(true); 
        t3.start();        
    } 
}```

2.
```java
// Java program to demonstrate the usage of exception in Daemon() Thread
public class DaemonThread extends Thread{
    public void run(){
        System.out.println("Thread name: " + Thread.currentThread().getName());
        System.out.println("Check if its DaemonThread: "+ Thread.currentThread().isDaemon());
    }
 
    public static void main(String[] args){
        DaemonThread t1 = new DaemonThread();
        DaemonThread t2 = new DaemonThread();
        t1.start();
         
        // Exception as the thread is already started
        t1.setDaemon(true);
         
        t2.start();
    }
}```

**Daemon vs User Threads:**
1. **Priority:** When the only remaining threads in a process are daemon threads, the interpreter exits. This makes sense because when only daemon threads remain, there is no other thread for which a daemon thread can provide a service.
2. **Usage:** Daemon thread is to provide services to user thread for background supporting task.


## Producer-Consumer solution using threads

## CountDownLatch in Java

## Deadlock

<img src="https://cdncontribute.geeksforgeeks.org/wp-content/uploads/22-2.png">

```java
// Java program to illustrate Deadlock in multithreading.
class Util{
    static void sleep(long millis){
        try{
            Thread.sleep(millis);
        }catch (InterruptedException e){
            e.printStackTrace();
        }
    }
}
 
// This class is shared by both threads
class Shared{
    synchronized void test1(Shared s2){// first synchronized method
        System.out.println("test1-begin");
        Util.sleep(1000);
        s2.test2(this);// taking object lock of s2 enters into test2 method
        System.out.println("test1-end");
    }
    synchronized void test2(Shared s1){// second synchronized method
        System.out.println("test2-begin");
        Util.sleep(1000);
        s1.test1(this);// taking object lock of s1 enters into test1 method
        System.out.println("test2-end");
    }
}
 
class Thread1 extends Thread{
    private Shared s1;
    private Shared s2;
    public Thread1(Shared s1, Shared s2){// constructor to initialize fields
        this.s1 = s1;
        this.s2 = s2;
    }
    @Override // run method to start a thread
    public void run(){
        s1.test1(s2);// taking object lock of s1 enters into test1 method
    }
}

class Thread2 extends Thread{
    private Shared s1;
    private Shared s2;
    public Thread2(Shared s1, Shared s2){// constructor to initialize fields
        this.s1 = s1;
        this.s2 = s2;
    }
    @Override// run method to start a thread
    public void run(){
        s2.test2(s1);// taking object lock of s2 enters into test2 method
    }
}

public class GFG{
    public static void main(String[] args){// creating one object
        Shared s1 = new Shared();
        Shared s2 = new Shared();// creating second object
        Thread1 t1 = new Thread1(s1, s2);// creating first thread and starting it
        t1.start();
        Thread2 t2 = new Thread2(s1, s2);// creating second thread and starting it
        t2.start();
        Util.sleep(2000);// sleeping main thread
    }
}```

## Semaphore
```java
// java program to demonstrate use of semaphores Locks
import java.util.concurrent.*;
 
//A shared resource/class.
class Shared {
    static int count = 0;
}
 
class MyThread extends Thread{
    Semaphore sem;
    String threadName;
    public MyThread(Semaphore sem, String threadName) {
        super(threadName);
        this.sem = sem;
        this.threadName = threadName;
    }
 
    @Override
    public void run() {
        // run by thread A
        if(this.getName().equals("A")){
            System.out.println("Starting " + threadName);
            try{
                // First, get a permit.
                System.out.println(threadName + " is waiting for a permit.");
             
                // acquiring the lock
                sem.acquire();
             
                System.out.println(threadName + " gets a permit.");
         
                // Now, accessing the shared resource. 
                // Other waiting threads will wait, until this thread release the lock
                for(int i=0; i < 5; i++){
                    Shared.count++;
                    System.out.println(threadName + ": " + Shared.count);
         
                    // Now, allowing a context switch -- if possible. for thread B to execute
                    Thread.sleep(10);
                }
            } catch (InterruptedException exc) {
                    System.out.println(exc);
            }
         
            // Release the permit.
            System.out.println(threadName + " releases the permit.");
            sem.release();
        }
        // run by thread B
        else{
            System.out.println("Starting " + threadName);
            try{
                // First, get a permit.
                System.out.println(threadName + " is waiting for a permit.");
             
                // acquiring the lock
                sem.acquire();
             
                System.out.println(threadName + " gets a permit.");
         
                // Now, accessing the shared resource.
                // Other waiting threads will wait, until this thread release the lock
                for(int i=0; i < 5; i++){
                    Shared.count--;
                    System.out.println(threadName + ": " + Shared.count);
         
                    // Now, allowing a context switch -- if possible. for thread A to execute
                    Thread.sleep(10);
                }
            } catch (InterruptedException exc) {
                    System.out.println(exc);
            }
            // Release the permit.
            System.out.println(threadName + " releases the permit.");
            sem.release();
        }
    }
}

public class SemaphoreDemo {
    public static void main(String args[]) throws InterruptedException {
        // creating a Semaphore object with number of permits 1
        Semaphore sem = new Semaphore(1);
         
        // creating two threads with name A and B 
        // Note that thread A will increment the count and thread B will decrement the count
        MyThread mt1 = new MyThread(sem, "A");
        MyThread mt2 = new MyThread(sem, "B");
         
        // stating threads A and B
        mt1.start();
        mt2.start();
         
        // waiting for threads A and B 
        mt1.join();
        mt2.join();
         
        // count will always remain 0 after both threads will complete their execution
        System.out.println("count: " + Shared.count);
    }
}```

## Java.util.concurrent.CyclicBarrier in Java
CyclicBarrier is used to make threads wait for each other. It is used when different threads process a part of computation and when all threads have completed the execution, the result needs to be combined in the parent thread. In other words, a CyclicBarrier is used when multiple thread carry out different sub tasks and the output of these sub tasks need to be combined to form the final output. After completing its execution, threads call await() method and wait for other threads to reach the barrier. Once all the threads have reached, the barriers then give the way for threads to proceed.

<img src="https://contribute.geeksforgeeks.org/wp-content/uploads/cyclicbarrier.png">

```java
//JAVA program to demonstrate execution on Cyclic Barrier
 
import java.util.concurrent.TimeUnit;
import java.util.concurrent.TimeoutException;
import java.util.concurrent.BrokenBarrierException;
import java.util.concurrent.CyclicBarrier;
 
class Computation1 implements Runnable{
    public static int product = 0;
    public void run(){
        product = 2 * 3;
        try{
            Tester.newBarrier.await();
        }catch (InterruptedException | BrokenBarrierException e){
            e.printStackTrace();
        }
    }
}

class Computation2 implements Runnable{
    public static int sum = 0;
    public void run(){
        // check if newBarrier is broken or not
        System.out.println("Is the barrier broken? - " + Tester.newBarrier.isBroken());
        sum = 10 + 20;
        try{
            Tester.newBarrier.await(3000, TimeUnit.MILLISECONDS);
            // number of parties waiting at the barrier
            System.out.println("Number of parties waiting at the barrier "+
            "at this point = " + Tester.newBarrier.getNumberWaiting());
        }catch (InterruptedException | BrokenBarrierException e) {
            e.printStackTrace();
        }catch (TimeoutException e){
            e.printStackTrace();
        }
    }
}

public class Tester implements Runnable{
    public static CyclicBarrier newBarrier = new CyclicBarrier(3);
     
    public static void main(String[] args){
        // parent thread
        Tester test = new Tester();
         
        Thread t1 = new Thread(test);
        t1.start();
    }
    public void run(){
        System.out.println("Number of parties required to trip the barrier = "+
        newBarrier.getParties());
        System.out.println("Sum of product and sum = " + (Computation1.product + 
        Computation2.sum));
         
        // objects on which the child thread has to run
        Computation1 comp1 = new Computation1();
        Computation2 comp2 = new Computation2();
         
        // creation of child thread
        Thread t1 = new Thread(comp1);
        Thread t2 = new Thread(comp2);
         
        // moving child thread to runnable state
        t1.start();
        t2.start();
 
        try{
            Tester.newBarrier.await();
        }catch (InterruptedException | BrokenBarrierException e){
            e.printStackTrace();
        }
         
        // barrier breaks as the number of thread waiting for the barrier at this point = 3
        System.out.println("Sum of product and sum = " + (Computation1.product + 
        Computation2.sum));
                 
        // Resetting the newBarrier
        newBarrier.reset();
        System.out.println("Barrier reset successful");
    }
}```

## Callable and Future in Java
There are two ways of creating threads – one by extending the Thread class and other by creating a thread with a Runnable. However, one feature lacking in  Runnable is that we cannot make a thread return result when it terminates, i.e. when run() completes. For supporting this feature, the Callable interface is present in Java.

**Callable vs Runnable:**
1. For implementing Runnable, the run() method needs to be implemented which does not return anything, while for a Callable, the call() method needs to be implemented which returns a result on completion. Note that a thread can’t be created with a Callable, it can only be created with a Runnable.
2. Another difference is that the call() method can throw an exception whereas run() cannot.

1.

```java
// Java program to illustrate Callable to return a random number
import java.util.Random;
import java.util.concurrent.Callable;
import java.util.concurrent.FutureTask;
 
class CallableExample implements Callable{
    public Object call() throws Exception{
        // Create random number generator
        Random generator = new Random();
 
        Integer randomNumber = generator.nextInt(5);
 
        // To simulate a heavy computation, we delay the thread for some random time
        Thread.sleep(randomNumber * 1000);
 
        return randomNumber;
    }
}```

#### Future
When the call() method completes, answer must be stored in an object known to the main thread, so that the main thread can know about the result that the thread returned. How will the program store and obtain this result later? For this, a Future object can be used. Think of a Future as an object that holds the result – it may not hold it right now, but it will do so in the future (once the Callable returns). Thus, a Future is basically one way the main thread can keep track of the progress and result from other threads. To implement this interface, 5 methods have to be overridden, but as the example below uses a concrete implementation from the library, only the important methods are listed here.

Observe that Callable and Future do two different things – Callable is similar to Runnable, in that it encapsulates a task that is meant to run on another thread, whereas a Future is used to store a result obtained from a different thread. In fact, the Future can be made to work with Runnable as well, which is something that will become clear when Executors come into the picture.

1.
```java
// Java program to illustrate Callable and FutureTask for random number generation
import java.util.Random;
import java.util.concurrent.Callable;
import java.util.concurrent.FutureTask;
 
class CallableExample implements Callable{
      public Object call() throws Exception{
            Random generator = new Random();
            Integer randomNumber = generator.nextInt(5);
            Thread.sleep(randomNumber * 1000);
            return randomNumber;
      }
 
}
public class CallableFutureTest{
      public static void main(String[] args) throws Exception{
            // FutureTask is a concrete class that implements both Runnable and Future
            FutureTask[] randomNumberTasks = new FutureTask[5];
            for (int i = 0; i < 5; i++){
                  Callable callable = new CallableExample();
                  // Create the FutureTask with Callable
                  randomNumberTasks[i] = new FutureTask(callable);
                  // As it implements Runnable, create Thread with FutureTask
                  Thread t = new Thread(randomNumberTasks[i]);
                  t.start();
            }

            for (int i = 0; i < 5; i++){
                  // As it implements Future, we can call get()
                  System.out.println(randomNumberTasks[i].get());

                  // This method blocks till the result is obtained The get method can throw checked exceptions
                  // like when it is interrupted. This is the reason for adding the throws clause to main
            }
      }
}```


Here is the code using only Runnable.
```java
// Java program to illustrate Runnable for random number generation
import java.util.Random;
import java.util.concurrent.Callable;
import java.util.concurrent.FutureTask;
 
class RunnableExample implements Runnable{
    // Shared object to store result
    private Object result = null;
    public void run(){
        Random generator = new Random();
        Integer randomNumber = generator.nextInt(5);
 
        // As run cannot throw any Exception
        try{
            Thread.sleep(randomNumber * 1000);
        }catch (InterruptedException e){
            e.printStackTrace();
        }
 
        // Store the return value in result when done
        result = randomNumber;
 
        // Wake up threads blocked on the get() method
        synchronized(this){
            notifyAll();
        }
    }
 
    public synchronized Object get() throws InterruptedException{
        while (result == null)
            wait();
        return result;
    }
}
 
// Code is almost same as the previous example with a few changes made to use Runnable instead of Callable
public class RunnableTest{
    public static void main(String[] args) throws Exception{
        RunnableExample[] randomNumberTasks = new RunnableExample[5];
 
        for (int i = 0; i < 5; i++){
            randomNumberTasks[i] = new RunnableExample();
            Thread t = new Thread(randomNumberTasks[i]);
            t.start();
        }
        for (int i = 0; i < 5; i++)
            System.out.println(randomNumberTasks[i].get());
    }
}```